举例1：大模型分析工具的调用

In [2]:
import os
import dotenv

from langchain_community.chat_models import ChatTongyi
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 原始工具实例
tools = [MoveFileTool()]

# 将工具转换为 OpenAI 风格的 function dict
functions = [convert_to_openai_function(t) for t in tools]
# ⚠️ DashScope 要求每个 tools 项包含 type 与 function 字段：
dashscope_tools = [{"type": "function", "function": f} for f in functions]

messages = [
    HumanMessage(content="将文件a移动到桌面")
]

# 调用模型 —— 传入 dashscope 风格的 tools
response = chat_model.invoke(
    messages,
    tools=dashscope_tools,
)

print(response)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


content='' additional_kwargs={'tool_calls': [{'function': {'arguments': '{"source_path": "a", "destination_path": "~/Desktop/a"}', 'name': 'move_file'}, 'id': 'call_bdf91e2c6da34eafb652bf8d', 'index': 0, 'type': 'function'}]} response_metadata={'model_name': 'qwen3-max', 'finish_reason': 'tool_calls', 'request_id': '8093ee8e-0c45-44fc-8d55-6d0e478754c1', 'token_usage': {'input_tokens': 301, 'output_tokens': 36, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 337}} id='run--9a6b4afa-577e-486f-95eb-cba69e6f02e5-0' tool_calls=[{'name': 'move_file', 'args': {'source_path': 'a', 'destination_path': '~/Desktop/a'}, 'id': 'call_bdf91e2c6da34eafb652bf8d', 'type': 'tool_call'}]


In [3]:
messages = [
    HumanMessage(content="帮我看一下北京明天的天气")
]

# 调用模型 —— 传入 dashscope 风格的 tools
response = chat_model.invoke(
    messages,
    tools=dashscope_tools,
)

print(response)

content='我无法直接获取实时天气信息。建议您通过天气预报网站、手机天气应用或搜索引擎查询“北京明天天气”来获取最新信息。' additional_kwargs={} response_metadata={'model_name': 'qwen3-max', 'finish_reason': 'stop', 'request_id': 'e63cfd15-0f19-488a-b779-cc8b6689c4b6', 'token_usage': {'input_tokens': 302, 'output_tokens': 31, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 333}} id='run--5dd6957e-9398-49c4-b311-069b8357c646-0'


通过上面的测试发现，得到的AIMessage的核心属性如下：

1、如果分析出需要调用对应的工具：

content：信息为空，因为大模型需要调用工具，所以就不会直接返回信息给用户

additional_kwargs：包含了工具调用的信息，主要是function_call字段，里面包含了要调用的工具名称与参数

2、如果没有分析出需要调用对应的工具：

content：直接返回给用户的信息

additional_kwargs：为空

举例2：如何调用大模型具体分析出来的工具呢

说明：

1、大模型与Agent的核心区别，是否涉及到工具的调用

2、针对于大模型：仅能分析出工具的调用，但是函数不能真正的执行

   针对于Agent：除了分析出要调用的工具之外，还可以执行具体的函数

In [4]:
import os
import dotenv

from langchain_community.chat_models import ChatTongyi
from langchain_community.tools import MoveFileTool
from langchain_core.messages import HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function

dotenv.load_dotenv()

chat_model = ChatTongyi(
    model="qwen3-max",
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

# 原始工具实例
tools = [MoveFileTool()]

# 将工具转换为 OpenAI 风格的 function dict
functions = [convert_to_openai_function(t) for t in tools]
# ⚠️ DashScope 要求每个 tools 项包含 type 与 function 字段：
dashscope_tools = [{"type": "function", "function": f} for f in functions]

messages = [
    HumanMessage(content="将当前目录下的文件a.txt移动到D:\\Desktop")
]

# 调用模型 —— 传入 dashscope 风格的 tools
response = chat_model.invoke(
    messages,
    tools=dashscope_tools,
)

print(response)

content='' additional_kwargs={'tool_calls': [{'function': {'arguments': '{"source_path": "a.txt", "destination_path": "D:\\\\Desktop\\\\a.txt"}', 'name': 'move_file'}, 'id': 'call_30d6f83a051948e7aa27a37e', 'index': 0, 'type': 'function'}]} response_metadata={'model_name': 'qwen3-max', 'finish_reason': 'tool_calls', 'request_id': 'bf4af4d3-0c71-4a65-9c6e-46e76975ba6f', 'token_usage': {'input_tokens': 307, 'output_tokens': 39, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 346}} id='run--32db9bb5-b812-4567-ab15-9a5464d61e26-0' tool_calls=[{'name': 'move_file', 'args': {'source_path': 'a.txt', 'destination_path': 'D:\\Desktop\\a.txt'}, 'id': 'call_30d6f83a051948e7aa27a37e', 'type': 'tool_call'}]


In [5]:
# 判断模型是否需要调用工具
if response.tool_calls:
    for call in response.tool_calls:
        name = call["name"]
        args = call["args"]   # 已经是 dict

        print("模型请求调用工具：", name, args)

        # 在 tools 列表中查找对应名称的工具
        tool_to_call = None
        for tool in tools:
            if tool.name == name:
                tool_to_call = tool
                break

        if tool_to_call is None:
            print(f"未找到名称为 {name} 的工具")
            continue

        # 执行工具
        result = tool_to_call.run(args)
        print("工具执行结果：", result)
else:
    print("模型未请求工具，输出：", response.content)


模型请求调用工具： move_file {'source_path': 'a.txt', 'destination_path': 'D:\\Desktop\\a.txt'}
工具执行结果： Error: no such file or directory a.txt
